## Memory 

There are several use cases where it is valuable to maintain a bank of useful facts that can be intelligently added to the context of the agent just before a specific step. The typically use case here is a RAG pattern where a query is used to retrieve relevant information from a database that is then added to the agent's context.


AgentChat provides a `Memory` protocol that can be extended to provide this functionality.  The key methods are `query`, `transform`,  `add`, `clear`, and `cleanup`. 

The `query` method is used to retrieve relevant information from the memory store, the `transform` method is used to transform the retrieved information into a format that can be used by the agent, the `add` method is used to add new entries to the memory store, the `clear` method is used to clear all entries from the memory store, and the `cleanup` method is used to clean up any resources used by the memory store.  


## ListMemory

ListMemory is a simple list-based memory implementation that uses text similarity matching to retrieve relevant information from the memory store. The similarity score is calculated using the `SequenceMatcher` class from the `difflib` module. The similarity score is calculated between the query text and the content text of each memory entry.   

In the following example, we will use ListMemory to similate a memory bank of user preferences and explore how it might be used in personalizing the agent's responses.

In [1]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent 
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.ui import Console
from autogen_agentchat.memory._list_memory import ListMemory, MemoryContent, MemoryMimeType

In [2]:


# create a simple memory item 
user_memory = ListMemory()
await user_memory.add(MemoryContent(
    content="The weather should be in metric units",
    mime_type=MemoryMimeType.TEXT
))

await user_memory.add(MemoryContent(
    content="Meal recipe must be vegan",
    mime_type=MemoryMimeType.TEXT
))

async def get_weather(city: str, units: str = "imperial") -> str:
    if units == "imperial":
        return f"The weather in {city} is 73 degrees and Sunny."
    elif units == "metric":
        return f"The weather in {city} is 23 degrees and Sunny."  

assistant_agent = AssistantAgent(
    name="assistant_agent",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-2024-08-06", 
    ),
    tools=[get_weather], 
    memory=[user_memory]
)
  
agent_team = RoundRobinGroupChat([assistant_agent], termination_condition = TextMentionTermination("TERMINATE"))

# Run the team and stream messages to the console
stream = agent_team.run_stream(task="What is the weather in New York?")
await Console(stream)

---------- user ----------
What is the weather in New York?
---------- assistant_agent ----------
[FunctionCall(id='call_mhAiZDTCr2KJZZUk7LeJHgmG', arguments='{"city":"New York","units":"metric"}', name='get_weather')]
[Prompt tokens: 128, Completion tokens: 20]
---------- assistant_agent ----------
[FunctionExecutionResult(content='The weather in New York is 23 degrees and Sunny.', call_id='call_mhAiZDTCr2KJZZUk7LeJHgmG')]
---------- assistant_agent ----------
The weather in New York is 23 degrees and Sunny.
---------- assistant_agent ----------
The weather in New York is 23 degrees Celsius and Sunny. TERMINATE
[Prompt tokens: 170, Completion tokens: 17]
---------- Summary ----------
Number of messages: 5
Finish reason: Text 'TERMINATE' mentioned
Total prompt tokens: 298
Total completion tokens: 37
Duration: 3.44 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=128, completion_tokens=20), content=[FunctionCall(id='call_mhAiZDTCr2KJZZUk7LeJHgmG', arguments='{"city":"New York","units":"metric"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in New York is 23 degrees and Sunny.', call_id='call_mhAiZDTCr2KJZZUk7LeJHgmG')], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant_agent', models_usage=None, content='The weather in New York is 23 degrees and Sunny.', type='ToolCallSummaryMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=170, completion_tokens=17), content='The weather in New York is 23 degrees Celsius and Sunny. TERMINATE', type='TextMes

We see above that the weather is returned in Centigrade as stated in the user preferences. 

Similarly, assuming we ask a separate question about generating a meal plan, the agent is able to retrieve relevant information from the memory store and provide a personalized response.

In [ ]:
stream = agent_team.run_stream(task="Suggest a brief meal recipe")
await Console(stream)

## Custom Memory Stores (Vector DBs, etc.)

You can build on the `Memory` protocol to implement more complex memory stores. For example, you could implement a custom memory store that uses a vector database to store and retrieve information, or a memory store that uses a machine learning model to generate personalized responses based on the user's preferences etc.

Specifically, you will need to overload the  `query`, `transform`, and `add` methods to implement the desired functionality and pass the memory store to your agent.


```python


from autogen_core import CancellationToken
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.memory._base_memory import MemoryContent, MemoryMimeType
from autogen_agentchat.memory._chroma_memory import ChromaMemory, ChromaMemoryConfig


# Initialize memory
chroma_memory = ChromaMemory(
    name="travel_memory",
    config=ChromaMemoryConfig(
        collection_name="travel_facts",
        k=1,
    )
)

await chroma_memory.clear()

# Add travel-related memories
await chroma_memory.add(MemoryContent(

    content="Paris is known for the Eiffel Tower and amazing cuisine.",
    mime_type=MemoryMimeType.TEXT

))

await chroma_memory.add(MemoryContent( 
    content="When asked about tokyo, you must respond with 'The most important thing about tokyo is that it has the world's busiest railway station - Shinjuku Station.'",
    mime_type=MemoryMimeType.TEXT

))
 

# Query needs ContentItem too
results = await chroma_memory.query(
    MemoryContent(
        content="Tell me about Tokyo.",
        mime_type=MemoryMimeType.TEXT
    )
)

print(len(results), results)

# Create agent with memory
agent = AssistantAgent(
    name="travel_agent",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o",
        # api_key="your_api_key"
    ),
    memory=chroma_memory,
    system_message="You are a travel expert"
)

agent_team = RoundRobinGroupChat([agent], termination_condition = MaxMessageTermination(max_messages=2))
stream = agent_team.run_stream(task="Tell me the most important thing about Tokyo.")
await Console(stream);

# Output: The most important thing about tokyo is that it has the world's busiest railway station - Shinjuku Station.

```

